In [1]:
!pip install gym
!pip install import-ipynb
!pip install ipynb
!pip install ortools


In [2]:
import gym
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



#instead of using OR-Tools, Gurobi will be used for the constraints 
#below commands needed to use methods from another ipynb file in Google Colab 
from google.colab import drive 
drive.mount('/content/gdrive/', force_remount=True)
%cd "gdrive/MyDrive/RLResearch"
!pip install import-ipynb 
import import_ipynb 
from MLBConstraintLibrary import MLBSchedulingGurobi


Mounted at /content/gdrive/
/content/gdrive/MyDrive/RLResearch
importing Jupyter notebook from MLBConstraintLibrary.ipynb


In [8]:
constraintLib = MLBSchedulingGurobi() #created constraintlib that has methods for diff constraints
constraintLib.test()
constraintLib.setFridayAndSaturdayHomeGames(4)

tested successfully


<p align = "justify">
Here we are setting up the AI Gym framework for the baseball scheduling problem modelled as a Markov Decision Process. We will set a maximum number of constraints $\textbf{M}$ here. Until we can figure out how to compute $\textbf{M}$ from the problem, we will temporarily set $\textbf{M}$ to equal 4. 
</p></br>

<p align = "justify">
For this simplified problem we will define the following variables:
$$c_{1},c_{2},c_{3},c_{4}$$ to represent 1,2,3,4 constraints applied to the sports scheduling problem respectively. Generally, we define $$c_{1},c_{2},..,c_{\textbf{M}}$$ where $\textbf{M}$ is the maximum number of constraints (depending on the optimization problem). For the general case, $c_{1}$ means 1 constraint applied to the sports scheduling problem, $c_{2}$ means 2 constraints applied, and so on until $c_{\textbf{M}}$ means $\textbf{M}$ constraints applied.
</p></br>

<p align = "justify">
Alternatively, we will use the following variables:
$$\tilde{c_{1}},\tilde{c_{2}},\tilde{c_{3}},\tilde{c_{4}}$$ to denote the 1,2,3,4 constraints applied to the sports scheduling problem where at least one of the constraints have been modified, substituted, or both.
Likewise, for the general case $\forall{i}$ $1 \leq i \leq \textbf{M}$, we will define $\tilde{c_{i}}$  to be $i$ constraints applied to the sports scheduling problem where at least one of the constraints are modified or substituted or both.
</p></br>

<p align = "justify">
Thus, the states in the Markov Decision Process (MDP) is the following set $\textbf{$S_{4}$}$ for the simplied problem where $\textbf{M}$ = 4:

Thus $S_{4} = \{c_{1},c_{2},c_{3},c_{4},\tilde{c_{1}},\tilde{c_{2}},\tilde{c_{3}},\tilde{c_{4}}\}$.

For the general case, the states of the MDP (call the states the set $S$) are as follows: $S = \{c_{i} \mid 1 \leq i \leq \textbf{M}\} \cup \{\tilde{c_{i}} \mid 1 \leq i \leq \textbf{M}\}$. 
</p></br>

<p align = "justify">
For $s \in S_{4}$, we will define the action space (i.e. we are here defining the action space for each state in the MDP). Note that the three actions are $\{"add","modify\_sub","delete"\}$ corresponding to add constraint, modify or substitute constraint, and delete constraint respectively. This is equivalent to saying that  $\forall s \in S_{4}, A(s) = \{"add","modify\_sub","delete"\}$. Note $modify\_sub$ means we either modify, substitute a constraint or both. 
</p></br>

<p align = "justify">
The transition graph where $\textbf{M}$ = 4 is defined in the following figure
</p></br>

<p align = "justify">
Note that our AI gym program is based off the Markov Decsion Process in Figure 1 below

<center>
<img src = 'https://raw.githubusercontent.com/pxu23/UmpireSchedulingProject/main/Research_Baseball/sports_schedule_mdp.PNG' width = "400" height = "300"></img>
<br>
        $\textbf{Figure 1}$
</center>

</p></br>

In [ ]:
## maximum number of constraints (temporarily we are hard coding it until we figure out a way to compute the maximum number of constraints)
## we can perhaps write a function that computes the maximum number of constraints depending on the problem description
class MLBSchedulingEnv(gym.Env):
  def __init__(self):
    super(MLBSchedulingEnv, self).__init__()

    """A sports scheduling environment for OpenAI gym"""
    metadata = {'render.modes': ['console']}

    ## action space for each state
    self.actions = np.array(['add','modify','delete'])

    ## defining the maximum number of constraints here (here we hard-code to 4)
    self.max_constraints = 4

    ## the states of the Markov Decision Process (stored as a list)
    self.states = np.array(['c1','c2','c3','c4','c1_modify','c2_modify_sub','c3_modify_sub','c4_modify_sub'])

    ## reward dictionary
    self.rewards = {}

    ## number of constraints we currently have
    self.numConstraints = 1 ## intially 1 for say

    ## boolean demonstrating if at least one of the constraints is modified
    self.constraintsModifySub = False ## initially none of the constraints are modified

    self.initialState = 'c1' ## assume we initially have one constraint where none of the constraints are modified
    self.currState = self.initialState

    self.constraints = ['numInterDivisionalGames', 'numIntraDivisionalGames', 'playInEverySlot','setHomeAwayPerGame','setNumHomeGameMinimum', 'setFridayAndSaturdayHomeGames', 'noBacktoBackSeries']

  ## Here we will take a step forward.
  ## As we have seen before, the actions for each state include add, delete, and modify
  def step(self,action):
    if action == 'add':         ## we add a constraint to the Markov Decision Process

      ## first check that adding a constraint will not exceed the maximum number of constraints
      if self.numConstraints == self.max_constraints:
        ## display an error message that we cannot add another constraint
        print("ERROR: Maximum constraint reached - cannot add one more")
        return ## exit the code here

      self.numConstraints += 1 ## the number of constraints increase by 1

      #add constraints here 
      #choose type of constraint by random 
      #need to confirm random number generator for each type of constraint in th emeeting 
      constraint_type = random.choice(self.constraints)
      if constraint_type == 'numInterDivisionalGames':
        #choose a random number of inter-divisional games, in general MLB has 20 inter-divisional games 
        numInterDivisionalGames = random.randint(15,20) #reasonable range  
        constraintLib.numInterDivisionalGames(numInterDivisionalGames)
      elif constraint_type == 'numIntraDivisionalGames': 
        #in general team plays 7 games against each of the other 6 teams from other divisions
        numIntraDivisionGames = random.randint(3,7)
        constraintLib.numIntraDivisionalGames(numIntraDivisionGames)
      elif constraint_type == 'playInEverySlot':
        constraintLib.playInEverySlot()
      elif constraint_type == 'setHomeAwayPerGame': 
        constraintLib.setHomeAwayPerGame()
      elif constraint_type == 'SetNumHomeGameMinimum': 
        #each team in MLB plays about 3 to 4 
        numMinHomeGames = random.randint(3,4)
        constraintLib.setNumHomeGamesMinimum(numMinHomeGames)
      elif constraint_type == 'setFridayAndSaturdayHomeGames': 
        numSatFriGames = random.randint(1,2)
        constraintLib.setFridayAndSaturdayHomeGames(numSatFriGames)
      else: #no back to back series 
        constraintLib.noBacktoBackSeries()
        

      ## updates the current state of the Markov decision process
      if self.constraintsModifySub == True:  ## at least one constraint being modified
        self.currState = self.states[self.max_constraints + self.numConstraints - 1]
      else: ## no constraint being modified
        self.currState = self.states[self.numConstraints - 1]

    elif action == 'modify': 
      if self.constraintsModifySub == False:
        self.constraintsModifySub = True
        self.currState = self.states[self.numConstraints - 1 + self.max_constraints]
      else:
        self.constraintsModifySub = False
        self.currState = self.states[self.numConstraints - 1]

    elif action == 'delete':
      #Question - deleting constraint also by random? 
      ## again check to make sure that if we delete a constraint, we will still have at least one constraint left
      if self.numConstraints == 1:
        print('ERROR: Deleting the constraint will result in having no constraints for this problem')
        return

      #call constraint Lib, delete constraint by random 
      constraintLib.deleteConstraint() #this method will choose one constraint & delete from model 

      self.numConstraints -= 1

      ## updates the current state of the Markov decision process
      if self.constraintsModifySub == True:  ## at least one constraint being modified
        self.currState = self.states[self.max_constraints + self.numConstraints - 1]
      else: ## no constraint being modified
        self.currState = self.states[self.numConstraints - 1]

    else:
      raise ValueError("Received invalid action={} which is not part of the action space".format(action))

  def reset(self):
    self.numConstraints = 1 ## back to having one constraints
    self.currState = self.initialState ## back to the start state
    self.constraintsModifySub = False  ## back to having no constraints modified
    print('reset successful')

  def render(self):
    print('maximum number of constraint for this problem is ', self.max_constraints)
    print('number of constraints is ',self.numConstraints)
    ## simulate taking 100 random actions 
    for i in range(100):
      ## agent randomly chooses an action from the list of actions
      action = random.choice(self.actions)
      print('iteration is ', i)
      print('action chosen is ',action)
      print('current state is ',self.currState)
      self.step(action) ## calling the step function that takes the randomly chosen action
      print('next state is ',self.currState)
      print('number of constraint is ',self.numConstraints)
      print('any modified constraints ? ',self.constraintsModifySub)
      print()

  ## here we will add multiple constraints
  def addMultipleConstraint(self,numConstraintToAdd):
    if self.numConstraints + numConstraintToAdd > self.max_constraints:
      print('ERROR: cannot add ',numConstraintToAdd,' constraints')
      return
    else:
      for i in range(numConstraintToAdd):
        self.step('add')
      print('successfully added ',numConstraintToAdd, ' constraints')
  
  ## here we will remove multiple constraints
  def removeMultipleConstraints(self,numConstraintsToRemove):
    if self.numConstraints - numConstraintsToRemove < 1:
      print('ERROR: cannot remove ', numConstraintsToRemove, ' constraints')
      return
    else:
      for i in range(numConstraintsToRemove):
        self.step('delete')
      print('successfully removed ',numConstraintsToRemove,' constraints')

<p align = "justify">
  Below, we will call the AI Gym framework we just wrote
</p>

In [ ]:
## creates a new sports scheculing agent/environment
scheduling_env =  MLBSchedulingEnv()

## calls the render() method that simulates taking random actions
scheduling_env.render()

## resets the environment/agent
scheduling_env.reset()

## try adding multiple constraints to the scheduling environment
scheduling_env.addMultipleConstraint(4) ## tune the parameter to see what happens

## try removing multiple constraints from the scheduling environment
scheduling_env.removeMultipleConstraints(3)



maximum number of constraint for this problem is  4
number of constraints is  1
iteration is  0
action chosen is  add
current state is  c1
next state is  c2
number of constraint is  2
any modified constraints ?  False

iteration is  1
action chosen is  add
current state is  c2
next state is  c3
number of constraint is  3
any modified constraints ?  False

iteration is  2
action chosen is  modify
current state is  c3
next state is  c3_modify_sub
number of constraint is  3
any modified constraints ?  True

iteration is  3
action chosen is  add
current state is  c3_modify_sub
next state is  c4_modify_sub
number of constraint is  4
any modified constraints ?  True

iteration is  4
action chosen is  add
current state is  c4_modify_sub
ERROR: Maximum constraint reached - cannot add one more
next state is  c4_modify_sub
number of constraint is  4
any modified constraints ?  True

iteration is  5
action chosen is  delete
current state is  c4_modify_sub
next state is  c3_modify_sub
number of co

*italicized text*<p align = "justify">
Now we will not only focus on the number of constraints but also on the types of constraints. 
</p></br>